In [1]:
from datasets import Dataset, DatasetDict, load_dataset
import os
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)



In [2]:
ds = load_dataset('yelp_review_full')

In [3]:
ds['train'][0]

{'label': 4,
 'text': "dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank."}

In [4]:
ds['train'].features


{'label': ClassLabel(names=['1 star', '2 star', '3 stars', '4 stars', '5 stars'], id=None),
 'text': Value(dtype='string', id=None)}

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [6]:
import pandas as pd
ds_train = pd.DataFrame(ds['train'])
ds_test = pd.DataFrame(ds['test'])


In [7]:
ds_train.head
ds_test.head

<bound method NDFrame.head of        label                                               text
0          0  I got 'new' tires from them and within two wee...
1          0  Don't waste your time.  We had two different p...
2          0  All I can say is the worst! We were the only 2...
3          0  I have been to this restaurant twice and was d...
4          0  Food was NOT GOOD at all! My husband & I ate h...
...      ...                                                ...
49995      0  Just wanted to write a review to chip in with ...
49996      4  Great ambience. Great drinks. Great food. I lo...
49997      3  I have been to the other Monks locations so I ...
49998      1  Don't go here.  I know you might want to try i...
49999      0  Buffet was recently open after renovation so m...

[50000 rows x 2 columns]>

In [29]:
#rename column label to labels
ds_train.rename(columns={'label':'labels'}, inplace=True)

In [30]:


# assign the splits
train = Dataset.from_pandas(ds_train)
test = Dataset.from_pandas(ds_test)
# reconstruct both datasets into a Dataset Dict object
new_ds = DatasetDict(
    {
        'train': train,
        'test': test
    }
)
# view the resulting dataset dict object
new_ds

DatasetDict({
    train: Dataset({
        features: ['labels', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [31]:
new_ds_path = "../results/new_ds"
print("Saving DatasetDict (new_ds) to disk...")
new_ds.save_to_disk(new_ds_path)

Saving DatasetDict (new_ds) to disk...


Saving the dataset (0/1 shards):   0%|          | 0/650000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [32]:
# Define model and tokenizer
model_name = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Path to save the tokenized dataset
tokenized_dataset_path = "../results/tokenized_dataset"

# Check if tokenized dataset exists
if os.path.exists(tokenized_dataset_path):
    print("Loading tokenized dataset from disk...")
    tokenized_ds = DatasetDict.load_from_disk(tokenized_dataset_path)
else:
    print("Tokenizing dataset...")

    # Tokenization function
    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

    # Assuming you already have the splits as DataFrames
    train = Dataset.from_pandas(ds_train)
    test = Dataset.from_pandas(ds_test)

    # Reconstruct the datasets into a DatasetDict object
    new_ds = DatasetDict({"train": train, "test": test})

    # Tokenize the dataset
    tokenized_ds = new_ds.map(tokenize_function, batched=True)

    # Remove unnecessary columns and format dataset
    tokenized_ds = tokenized_ds.remove_columns(["text"])
   
    tokenized_ds.set_format("torch")

    # Save tokenized dataset to disk
    tokenized_ds.save_to_disk(tokenized_dataset_path)
    print("Tokenized dataset saved to disk.")

Loading tokenized dataset from disk...


In [10]:
# Initialize data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",               # Directory to save results
    eval_strategy="epoch",         # Evaluate after every epoch
    save_strategy="epoch",               # Save model after every epoch
    learning_rate=2e-5,                  # Default learning rate (to be tuned)
    per_device_train_batch_size=16,      # Default batch size (to be tuned)
    per_device_eval_batch_size=16,       # Batch size for evaluation
    num_train_epochs=3,                  # Default number of epochs (to be tuned)
    weight_decay=0.01,                   # Regularization
    gradient_accumulation_steps=4,       # Accumulate gradients over 4 steps
    fp16=True,                           # Mixed-precision training
    logging_dir="./logs",                # Log directory
    logging_steps=10,                    # Log every 10 steps
    save_total_limit=2,                  # Limit saved checkpoints to 2
    disable_tqdm=False                   # Enable progress bar
)

In [12]:
# Define model initialization for hyperparameter search
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)


In [13]:
# Define hyperparameter search space
def hyperparameter_search(trial):
    return {
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 5e-5),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32]),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 2, 5),
        "weight_decay": trial.suggest_uniform("weight_decay", 0.01, 0.1),
    }

In [17]:
# Initialize the Trainer
trainer = Trainer(
    model_init=model_init,  # For hyperparameter search
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    data_collator=data_collator,  # Use the data collator for dynamic padding
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# Perform hyperparameter search
print("Starting hyperparameter search...")
best_run = trainer.hyperparameter_search(
    n_trials=5,  # Reduce for quicker results
    direction="minimize",  # Minimize evaluation loss
    compute_objective=lambda metrics: metrics["eval_loss"],
    hp_space=lambda trial: hyperparameter_search(trial),
)

# Print the best hyperparameters
print("Best Hyperparameters:")
print(best_run.hyperparameters)

[I 2024-12-18 01:57:32,626] A new study created in memory with name: no-name-68d38268-dbd2-41e3-8e7f-e8d372e28c92
/tmp/ipykernel_1293732/1311042639.py:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 5e-5),
/tmp/ipykernel_1293732/1311042639.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  "weight_decay": trial.suggest_uniform("weight_decay", 0.01, 0.1),
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this m

Starting hyperparameter search...


  0%|          | 0/40624 [00:00<?, ?it/s]

{'loss': 6.4667, 'grad_norm': 4.195240020751953, 'learning_rate': 2.744639372496024e-05, 'epoch': 0.0}
{'loss': 6.4117, 'grad_norm': 7.178688049316406, 'learning_rate': 2.7439635859759827e-05, 'epoch': 0.0}
{'loss': 6.1921, 'grad_norm': 10.30695915222168, 'learning_rate': 2.7432877994559412e-05, 'epoch': 0.0}
{'loss': 5.7433, 'grad_norm': 16.48085594177246, 'learning_rate': 2.7426120129358997e-05, 'epoch': 0.0}
{'loss': 5.3887, 'grad_norm': 26.727764129638672, 'learning_rate': 2.7420038050678627e-05, 'epoch': 0.0}
{'loss': 4.9027, 'grad_norm': inf, 'learning_rate': 2.7414631758518297e-05, 'epoch': 0.0}
{'loss': 4.9174, 'grad_norm': 35.01726150512695, 'learning_rate': 2.7407873893317882e-05, 'epoch': 0.0}
{'loss': 4.5049, 'grad_norm': 19.07581901550293, 'learning_rate': 2.7401116028117467e-05, 'epoch': 0.0}
{'loss': 4.7778, 'grad_norm': 29.106046676635742, 'learning_rate': 2.7394358162917055e-05, 'epoch': 0.0}
{'loss': 4.4462, 'grad_norm': 22.236589431762695, 'learning_rate': 2.73876002

  0%|          | 0/3125 [00:00<?, ?it/s]

...

{'eval_loss': 0.7664861679077148, 'eval_runtime': 26.0487, 'eval_samples_per_second': 1919.479, 'eval_steps_per_second': 119.967, 'epoch': 3.0}
{'loss': 2.5167, 'grad_norm': 26.120285034179688, 'learning_rate': 6.3646043943436045e-06, 'epoch': 3.0}
{'loss': 2.0994, 'grad_norm': 24.692306518554688, 'learning_rate': 6.361474209720409e-06, 'epoch': 3.0}
{'loss': 2.2528, 'grad_norm': 24.23455047607422, 'learning_rate': 6.358344025097214e-06, 'epoch': 3.0}
{'loss': 2.2338, 'grad_norm': 23.79096794128418, 'learning_rate': 6.355213840474017e-06, 'epoch': 3.0}
{'loss': 2.1664, 'grad_norm': 25.39236831665039, 'learning_rate': 6.352083655850822e-06, 'epoch': 3.0}
{'loss': 2.0069, 'grad_norm': 23.01955795288086, 'learning_rate': 6.3489534712276265e-06, 'epoch': 3.0}
{'loss': 2.2205, 'grad_norm': 17.304744720458984, 'learning_rate': 6.34582328660443e-06, 'epoch': 3.0}
{'loss': 2.3343, 'grad_norm': 32.650081634521484, 'learning_rate': 6.342693101981236e-06, 'epoch': 3.0}
{'loss': 1.976, 'grad_norm'

  0%|          | 0/3125 [00:00<?, ?it/s]

{'eval_loss': 0.8196930289268494, 'eval_runtime': 26.0538, 'eval_samples_per_second': 1919.108, 'eval_steps_per_second': 119.944, 'epoch': 4.0}


[I 2024-12-18 08:12:59,935] Trial 4 finished with value: 0.8196930289268494 and parameters: {'learning_rate': 2.5432124026539574e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 4, 'weight_decay': 0.05829680099089378}. Best is trial 0 with value: 0.7349283695220947.


{'train_runtime': 6294.9242, 'train_samples_per_second': 413.031, 'train_steps_per_second': 12.907, 'train_loss': 2.6820999547786344, 'epoch': 4.0}
Best Hyperparameters:
{'learning_rate': 2.7453151590160654e-05, 'per_device_train_batch_size': 8, 'num_train_epochs': 2, 'weight_decay': 0.026147972761883034}
